# Ten'up - Selenium

<img src = 'https://tenup.fft.fr/sites/all/themes/met/images/generic-img.png' >


Dans ce projet, nous utiliserons la bibliothèque Selenium afin de collecter les données du site internet suivant :

- [Ten'up](https://tenup.fft.fr//)
Nous collecterons les données correspondants au tournois par rapport à une ville.

Les informations que nous souhaitons collecter sont les suivantes :
- le nom du tournois
- la localisation
- date de début
- date de fin
- catégorie des participant
- la disctance par rapport au point de recherche
- le point de recherche
- la date et l'heure de la recherche

# 1 Configuration du projet

## 1.1 Configuration de venv

In [6]:
# !python3 -m venv env .\venv # commande pour mettre en place un env virtuel en python
# !source env/Scripts/activate # commande pour activer un env virtuel en python
# deactivate # command to stop the virtual environment
# pip install -r requirements.txt # commande pour installer toute les dépandances necessaire pour le projet

## 1.2 Installez la bibliothèque Selenium

In [5]:
!pip install selenium

## 1.3 Installez la bibliothèque webdriver-manager 

In [7]:
!pip install webdriver-manager 

# 2 Configuration Selenium

## 2.1 Importer la bibliothèque selenium

In [ ]:
from selenium import webdriver

## 2.2 Importer l'objet `webdriver` de la bibliothèque de `selenium`

In [8]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## 2.3 Importer la bibliothèque webdriver_manager

In [ ]:

from webdriver_manager.chrome import ChromeDriverManager

## 2.4 Initialiser l'url
`BASE_URL` est le lien pointant vers la recherche de tournois du site `Ten'up` `"https://tenup.fft.fr/recherche/tournois"`

In [ ]:
BASE_URL = "https://tenup.fft.fr/recherche/tournois"

## 2.5 Deffinir les options pour webdriver

In [ ]:
chrome_options = Options
chrome_options.add_experimental_option("detach", True)

In [ ]:
driver = Chrome(service=Service(ChromeDriverManager().install()),
                options=chrome_options)

# 3 Début de la recherche

## 3.1 Acceder à l'url

driver.get(BASE_URL)

## 3.2 Affichez l'adresse url courante ainsi que le titre de la page.

In [ ]:
driver.current_url

In [ ]:
driver.title

# 4 : Gestion des bases de données

## 4.1 Création du fichier utils.py

In [2]:
import sqlalchemy as db

class DataBase():
    def __init__(self, name_database='database'):
        self.name = name_database
        self.url = f"sqlite:///{name_database}.db"
        self.engine = db.create_engine(self.url)
        self.connection = self.engine.connect()
        self.metadata = db.MetaData()
        self.table = self.engine.table_names()
       
    
    def create_table(self, name_table, **kwargs):
        colums = [db.Column(k, v, primary_key = True) if 'id_' in k else db.Column(k, v) for k,v in kwargs.items()]
        db.Table(name_table, self.metadata, *colums)
        self.metadata.create_all(self.engine)
        print(f"Table : '{name_table}' are created succesfully")
        
    def read_table(self, name_table, return_keys=False):
        table = db.Table(name_table, self.metadata, autoload=True, autoload_with=self.engine)
        if return_keys:table.columns.keys() 
        else : return table
        
        
    def add_row(self, name_table, **kwarrgs):
        name_table = self.read_table(name_table)
        
        stmt = (
            db.insert(name_table).
            values(kwarrgs)
        )
        self.connection.execute(stmt)
        print(f'Row id added')
        
        
    def delete_row_by_id(self, table, id_):
        name_table = self.read_table(name_table) 
        
        stmt = (
            db.delete(name_table).
            where(students.c.id_ == id_)
            )
        self.connection.execute(stmt)
        print(f'Row id {id_} deleted')
        
    def select_table(self, name_table):
        name_table = self.read_table(name_table)       
        stm = db.select([name_table])
        return self.connection.execute(stm).fetchall()

## 4.2 - Importation de la base de donnée dans app.py

In [3]:
# Import du package sqlalchemy permetant de gérer les bases de données
import sqlalchemy as db

# Import de la class DataBase permettant de créer des bases de données
from utils import DataBase

## 4.3 - Instaciation de la base de données 'tenup.sqlite'

In [4]:
database_name = "tenup"
table_name = "tournois"
base = DataBase(database_name)

## 4.4 - Création d'une table nommée "tournois" dans la base de données 'tenup.sqlite'

In [5]:
base.create_table(table_name, 
                 name=db.String, # nom du tournois
                 location=db.String, # la localisation du tournois
                 start_date=db.String, # date de début        
                 end_date=db.String, # date de fin
                 category=db.String, # catégorie des participants
                 distance=db.String, # la disctance par rapport au point de recherche
                 search_location=db.String, # le point de recherche
                 search_time=db.String # la date et l'heure de la recherche
                 )

Table : 'tournois' are created succesfully


## 4.4 - Création d'un objet item contenant les données à stocker dans la base de données

In [6]:
item = {}
item['name'] = "test" # nom du tournois
item['location'] = "paris" # la localisation du tournois
item['start_date'] = "27/10/22" # date de début
item['end_date'] = "27/10/22" # date de fin
item['category'] = "MS" # catégorie des participants
item['distance'] = "2,7" # la disctance par rapport au point de recherche
item['search_location'] = "Massy, 91, Essonne, Île-de-France" # le point de recherche
item['search_time'] = "27/10/22" # la date et l'heure de la recherche
print(item)

{'name': 'test', 'location': 'paris', 'start_date': '27/10/22', 'end_date': '27/10/22', 'category': 'MS', 'distance': '2,7', 'search_location': 'Massy, 91, Essonne, Île-de-France', 'search_time': '27/10/22'}


## 4.5 - Ajout d'une ligne dans la base de données 'tenup.sqlite'

In [7]:
base.add_row(table_name,
                 name=item['name'], # nom du tournois
                 location=item['location'], # - la localisation du tournois
                 start_date=item['start_date'], # date de début        
                 end_date=item['end_date'], # date de fin
                 category=item['category'], # catégorie des participants
                 distance=item['distance'], # la disctance par rapport au point de recherche
                 search_location=item['search_location'], # le point de recherche
                 search_time=item['search_time'] # la date et l'heure de la recherche
            )


Row id added


## 4.6 - Afficher les colonnes de la table

In [11]:
data_read = base.read_table(table_name)
print('table:',data_read)
print('content:',data_read.columns.keys())

table: tournois
content: ['name', 'location', 'start_date', 'end_date', 'category', 'distance', 'search_location', 'search_time']


## 4.7 - Afficher le contenu de la table 'Table_Test'

In [14]:
print("table:",base.select_table(table_name))

table: [('test', 'paris', '27/10/22', '27/10/22', 'MS', '2,7', 'Massy, 91, Essonne, Île-de-France', '27/10/22')]
